In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,LearningRateScheduler,EarlyStopping

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        r = (band_1 - band_1.min()) / (band_1.max() - band_1.min())
        g = (band_2 - band_2.min()) / (band_2.max() - band_2.min())
        b = (band_3 - band_3.min()) / (band_3.max() - band_3.min())

        rgb = np.dstack((r, g, b))
        images.append(rgb)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(16, (3, 3), padding='same',input_shape=(75,75,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(16, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model
print('model model')


model model


In [5]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch<10:
        return 0.001
    elif epoch<30:
        return 0.0005
    elif epoch<50:
        return 0.0001
    else:
        return 0.00005

def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=5)

import pickle
with open('../features/cnn_1_aug_rescale_preprocess_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))

#pre 3219
# new 2965

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.67286, saving model to best_m.h5
 - 2s - loss: 0.6907 - acc: 0.5208 - val_loss: 0.6729 - val_acc: 0.5078
Epoch 2/100
Epoch 00002: val_loss did not improve
 - 1s - loss: 0.6069 - acc: 0.6554 - val_loss: 0.6806 - val_acc: 0.5857
Epoch 3/100
Epoch 00003: val_loss improved from 0.67286 to 0.58292, saving model to best_m.h5
 - 2s - loss: 0.5894 - acc: 0.6716 - val_loss: 0.5829 - val_acc: 0.6916
Epoch 4/100
Epoch 00004: val_loss improved from 0.58292 to 0.57786, saving model to best_m.h5
 - 2s - loss: 0.6405 - acc: 0.6058 - val_loss: 0.5779 - val_acc: 0.6978
Epoch 5/100
Epoch 00005: val_loss improved from 0.57786 to 0.57044, saving model to best_m.h5
 - 2s - loss: 0.5768 - acc: 0.6737 - val_loss: 0.5704 - val_acc: 0.7009
Epoch 6/100
Epoch 00006: val_loss did not improve
 - 2s - loss: 0.5842 - acc: 0.6747 - val_loss: 0.5705 - val_acc: 0.7196
Epoch 7/100
Epoch 00007: val_loss improved from 0.57044 to 0.56848, saving model to best_m.h5
 -

Epoch 00058: val_loss did not improve
 - 1s - loss: 0.3089 - acc: 0.8474 - val_loss: 0.2638 - val_acc: 0.8785
Epoch 59/100
Epoch 00059: val_loss did not improve
 - 1s - loss: 0.3179 - acc: 0.8568 - val_loss: 0.2620 - val_acc: 0.8816
Epoch 60/100
Epoch 00060: val_loss did not improve
 - 1s - loss: 0.3148 - acc: 0.8519 - val_loss: 0.2607 - val_acc: 0.8785
Epoch 61/100
Epoch 00061: val_loss did not improve
 - 1s - loss: 0.3318 - acc: 0.8391 - val_loss: 0.2629 - val_acc: 0.8816
Epoch 62/100
Epoch 00062: val_loss did not improve
 - 1s - loss: 0.3271 - acc: 0.8433 - val_loss: 0.2658 - val_acc: 0.8754
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 1s - loss: 0.3099 - acc: 0.8396 - val_loss: 0.2700 - val_acc: 0.8660
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 1s - loss: 0.3117 - acc: 0.8625 - val_loss: 0.2632 - val_acc: 0.8754
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 1s - loss: 0.3082 - acc: 0.8495 - val_loss: 0.2603 - val_acc: 0.8847
Epoch 66/100
Epoch 00066: val

Epoch 19/100
Epoch 00019: val_loss improved from 0.33560 to 0.33211, saving model to best_m.h5
 - 2s - loss: 0.3798 - acc: 0.8180 - val_loss: 0.3321 - val_acc: 0.8380
Epoch 20/100
Epoch 00020: val_loss improved from 0.33211 to 0.31171, saving model to best_m.h5
 - 2s - loss: 0.3246 - acc: 0.8515 - val_loss: 0.3117 - val_acc: 0.8442
Epoch 21/100
Epoch 00021: val_loss did not improve
 - 1s - loss: 0.3535 - acc: 0.8247 - val_loss: 0.3293 - val_acc: 0.8349
Epoch 22/100
Epoch 00022: val_loss did not improve
 - 1s - loss: 0.3484 - acc: 0.8287 - val_loss: 0.3295 - val_acc: 0.8380
Epoch 23/100
Epoch 00023: val_loss did not improve
 - 1s - loss: 0.3649 - acc: 0.8250 - val_loss: 0.3352 - val_acc: 0.8255
Epoch 24/100
Epoch 00024: val_loss did not improve
 - 1s - loss: 0.3499 - acc: 0.8344 - val_loss: 0.3577 - val_acc: 0.8380
Epoch 25/100
Epoch 00025: val_loss did not improve
 - 1s - loss: 0.3291 - acc: 0.8419 - val_loss: 0.3175 - val_acc: 0.8660
Epoch 26/100
Epoch 00026: val_loss did not improve


Epoch 81/100
Epoch 00081: val_loss did not improve
 - 1s - loss: 0.2901 - acc: 0.8672 - val_loss: 0.2930 - val_acc: 0.8629
Epoch 82/100
Epoch 00082: val_loss improved from 0.28974 to 0.28936, saving model to best_m.h5
 - 2s - loss: 0.2826 - acc: 0.8695 - val_loss: 0.2894 - val_acc: 0.8567
Epoch 83/100
Epoch 00083: val_loss improved from 0.28936 to 0.28835, saving model to best_m.h5
 - 2s - loss: 0.2955 - acc: 0.8488 - val_loss: 0.2883 - val_acc: 0.8536
Epoch 84/100
Epoch 00084: val_loss improved from 0.28835 to 0.28796, saving model to best_m.h5
 - 2s - loss: 0.2939 - acc: 0.8547 - val_loss: 0.2880 - val_acc: 0.8536
Epoch 85/100
Epoch 00085: val_loss improved from 0.28796 to 0.28712, saving model to best_m.h5
 - 1s - loss: 0.2869 - acc: 0.8625 - val_loss: 0.2871 - val_acc: 0.8536
Epoch 86/100
Epoch 00086: val_loss did not improve
 - 1s - loss: 0.3017 - acc: 0.8589 - val_loss: 0.2879 - val_acc: 0.8629
Epoch 87/100
Epoch 00087: val_loss did not improve
 - 1s - loss: 0.2821 - acc: 0.8656 

Epoch 40/100
Epoch 00040: val_loss did not improve
 - 1s - loss: 0.3190 - acc: 0.8459 - val_loss: 0.2737 - val_acc: 0.8847
Epoch 41/100
Epoch 00041: val_loss did not improve
 - 1s - loss: 0.3193 - acc: 0.8430 - val_loss: 0.2785 - val_acc: 0.8629
Epoch 42/100
Epoch 00042: val_loss did not improve
 - 1s - loss: 0.3296 - acc: 0.8326 - val_loss: 0.2781 - val_acc: 0.8723
Epoch 43/100
Epoch 00043: val_loss improved from 0.27289 to 0.26992, saving model to best_m.h5
 - 2s - loss: 0.3205 - acc: 0.8437 - val_loss: 0.2699 - val_acc: 0.8816
Epoch 44/100
Epoch 00044: val_loss did not improve
 - 1s - loss: 0.3106 - acc: 0.8594 - val_loss: 0.2797 - val_acc: 0.8723
Epoch 45/100
Epoch 00045: val_loss improved from 0.26992 to 0.26689, saving model to best_m.h5
 - 2s - loss: 0.3325 - acc: 0.8352 - val_loss: 0.2669 - val_acc: 0.8816
Epoch 46/100
Epoch 00046: val_loss did not improve
 - 1s - loss: 0.3144 - acc: 0.8453 - val_loss: 0.2788 - val_acc: 0.8816
Epoch 47/100
Epoch 00047: val_loss did not improve


 - 2s - loss: 0.6912 - acc: 0.5341 - val_loss: 0.6838 - val_acc: 0.5140
Epoch 2/100
Epoch 00002: val_loss improved from 0.68380 to 0.59227, saving model to best_m.h5
 - 2s - loss: 0.6317 - acc: 0.6526 - val_loss: 0.5923 - val_acc: 0.6885
Epoch 3/100
Epoch 00003: val_loss did not improve
 - 1s - loss: 0.6319 - acc: 0.6370 - val_loss: 0.6085 - val_acc: 0.6355
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 1s - loss: 0.5813 - acc: 0.6851 - val_loss: 0.6642 - val_acc: 0.5576
Epoch 5/100
Epoch 00005: val_loss improved from 0.59227 to 0.57235, saving model to best_m.h5
 - 2s - loss: 0.5949 - acc: 0.6911 - val_loss: 0.5724 - val_acc: 0.6760
Epoch 6/100
Epoch 00006: val_loss improved from 0.57235 to 0.57145, saving model to best_m.h5
 - 1s - loss: 0.5839 - acc: 0.6734 - val_loss: 0.5714 - val_acc: 0.6698
Epoch 7/100
Epoch 00007: val_loss did not improve
 - 1s - loss: 0.5722 - acc: 0.6789 - val_loss: 0.5893 - val_acc: 0.6604
Epoch 8/100
Epoch 00008: val_loss improved from 0.57145 to 0.541

Epoch 00063: val_loss did not improve
 - 1s - loss: 0.3131 - acc: 0.8339 - val_loss: 0.3696 - val_acc: 0.8224
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 1s - loss: 0.3274 - acc: 0.8445 - val_loss: 0.3794 - val_acc: 0.8193
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 1s - loss: 0.3390 - acc: 0.8326 - val_loss: 0.4115 - val_acc: 0.7819
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 1s - loss: 0.3219 - acc: 0.8461 - val_loss: 0.3850 - val_acc: 0.8006
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 1s - loss: 0.3395 - acc: 0.8466 - val_loss: 0.3985 - val_acc: 0.7882
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 1s - loss: 0.3133 - acc: 0.8466 - val_loss: 0.3996 - val_acc: 0.7788
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 1s - loss: 0.3244 - acc: 0.8313 - val_loss: 0.3887 - val_acc: 0.8069
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 1s - loss: 0.3034 - acc: 0.8430 - val_loss: 0.3747 - val_acc: 0.8100
Epoch 71/100
Epoch 00071: val

Epoch 26/100
Epoch 00026: val_loss improved from 0.35844 to 0.35366, saving model to best_m.h5
 - 2s - loss: 0.3598 - acc: 0.8211 - val_loss: 0.3537 - val_acc: 0.8500
Epoch 27/100
Epoch 00027: val_loss did not improve
 - 1s - loss: 0.3710 - acc: 0.8258 - val_loss: 0.4327 - val_acc: 0.7875
Epoch 28/100
Epoch 00028: val_loss did not improve
 - 1s - loss: 0.3616 - acc: 0.8203 - val_loss: 0.3874 - val_acc: 0.8031
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 1s - loss: 0.3722 - acc: 0.8078 - val_loss: 0.3623 - val_acc: 0.8406
Epoch 30/100
Epoch 00030: val_loss improved from 0.35366 to 0.34251, saving model to best_m.h5
 - 2s - loss: 0.3271 - acc: 0.8484 - val_loss: 0.3425 - val_acc: 0.8438
Epoch 31/100
Epoch 00031: val_loss did not improve
 - 1s - loss: 0.3347 - acc: 0.8446 - val_loss: 0.3633 - val_acc: 0.8375
Epoch 32/100
Epoch 00032: val_loss did not improve
 - 1s - loss: 0.3031 - acc: 0.8594 - val_loss: 0.3711 - val_acc: 0.8281
Epoch 33/100
Epoch 00033: val_loss did not improve


Epoch 92/100
Epoch 00092: val_loss did not improve
 - 1s - loss: 0.2997 - acc: 0.8672 - val_loss: 0.3477 - val_acc: 0.8438
Epoch 93/100
Epoch 00093: val_loss did not improve
 - 1s - loss: 0.2870 - acc: 0.8633 - val_loss: 0.3546 - val_acc: 0.8438
Epoch 94/100
Epoch 00094: val_loss did not improve
 - 1s - loss: 0.2867 - acc: 0.8672 - val_loss: 0.3499 - val_acc: 0.8469
Epoch 95/100
Epoch 00095: val_loss did not improve
 - 1s - loss: 0.2782 - acc: 0.8704 - val_loss: 0.3548 - val_acc: 0.8375
Epoch 96/100
Epoch 00096: val_loss did not improve
 - 1s - loss: 0.2913 - acc: 0.8578 - val_loss: 0.3499 - val_acc: 0.8313
Epoch 97/100
Epoch 00097: val_loss did not improve
 - 1s - loss: 0.2803 - acc: 0.8727 - val_loss: 0.3546 - val_acc: 0.8469
Epoch 98/100
Epoch 00098: val_loss did not improve
 - 1s - loss: 0.2892 - acc: 0.8641 - val_loss: 0.3498 - val_acc: 0.8469
Epoch 99/100
Epoch 00099: val_loss did not improve
 - 1s - loss: 0.2785 - acc: 0.8789 - val_loss: 0.3586 - val_acc: 0.8406
Epoch 100/100
Ep

In [6]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(32, (3, 3),input_shape=(75, 75, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))
    return model
print('model model')

model model


In [7]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=5)


with open('../features/cnn_2_aug_rescale_preprocess_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))

# this 2728

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.57310, saving model to best_m.h5
 - 2s - loss: 0.6710 - acc: 0.5575 - val_loss: 0.5731 - val_acc: 0.7321
Epoch 2/100
Epoch 00002: val_loss improved from 0.57310 to 0.53775, saving model to best_m.h5
 - 1s - loss: 0.5721 - acc: 0.6951 - val_loss: 0.5378 - val_acc: 0.7601
Epoch 3/100
Epoch 00003: val_loss improved from 0.53775 to 0.50752, saving model to best_m.h5
 - 1s - loss: 0.5201 - acc: 0.7422 - val_loss: 0.5075 - val_acc: 0.7819
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 1s - loss: 0.5595 - acc: 0.6935 - val_loss: 0.5531 - val_acc: 0.7072
Epoch 5/100
Epoch 00005: val_loss improved from 0.50752 to 0.50205, saving model to best_m.h5
 - 1s - loss: 0.5405 - acc: 0.7172 - val_loss: 0.5021 - val_acc: 0.7819
Epoch 6/100
Epoch 00006: val_loss improved from 0.50205 to 0.44124, saving model to best_m.h5
 - 1s - loss: 0.5025 - acc: 0.7609 - val_loss: 0.4412 - val_acc: 0.8100
Epoch 7/100
Epoch 00007: val_loss did not improve
 -

Epoch 61/100
Epoch 00061: val_loss did not improve
 - 1s - loss: 0.2711 - acc: 0.8730 - val_loss: 0.2707 - val_acc: 0.8879
Epoch 62/100
Epoch 00062: val_loss did not improve
 - 1s - loss: 0.2686 - acc: 0.8742 - val_loss: 0.2660 - val_acc: 0.8785
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 1s - loss: 0.2644 - acc: 0.8828 - val_loss: 0.2519 - val_acc: 0.8910
Epoch 64/100
Epoch 00064: val_loss improved from 0.25142 to 0.24828, saving model to best_m.h5
 - 1s - loss: 0.2608 - acc: 0.8844 - val_loss: 0.2483 - val_acc: 0.9003
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 1s - loss: 0.2532 - acc: 0.8804 - val_loss: 0.2525 - val_acc: 0.8910
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 1s - loss: 0.2840 - acc: 0.8664 - val_loss: 0.2550 - val_acc: 0.8879
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 1s - loss: 0.2504 - acc: 0.8812 - val_loss: 0.2576 - val_acc: 0.8847
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 1s - loss: 0.2548 - acc: 0.8781 - val_los

Epoch 00022: val_loss did not improve
 - 1s - loss: 0.3675 - acc: 0.8273 - val_loss: 0.3396 - val_acc: 0.8380
Epoch 23/100
Epoch 00023: val_loss did not improve
 - 1s - loss: 0.3121 - acc: 0.8584 - val_loss: 0.3378 - val_acc: 0.8380
Epoch 24/100
Epoch 00024: val_loss did not improve
 - 1s - loss: 0.3471 - acc: 0.8255 - val_loss: 0.3451 - val_acc: 0.8287
Epoch 25/100
Epoch 00025: val_loss did not improve
 - 1s - loss: 0.3279 - acc: 0.8425 - val_loss: 0.3336 - val_acc: 0.8287
Epoch 26/100
Epoch 00026: val_loss did not improve
 - 1s - loss: 0.3076 - acc: 0.8437 - val_loss: 0.3629 - val_acc: 0.8505
Epoch 27/100
Epoch 00027: val_loss did not improve
 - 1s - loss: 0.2982 - acc: 0.8539 - val_loss: 0.3746 - val_acc: 0.8131
Epoch 28/100
Epoch 00028: val_loss did not improve
 - 1s - loss: 0.3178 - acc: 0.8602 - val_loss: 0.3489 - val_acc: 0.8162
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 1s - loss: 0.3276 - acc: 0.8497 - val_loss: 0.3331 - val_acc: 0.8287
Epoch 30/100
Epoch 00030: val

Epoch 83/100
Epoch 00083: val_loss did not improve
 - 1s - loss: 0.2637 - acc: 0.8820 - val_loss: 0.2852 - val_acc: 0.8629
Epoch 84/100
Epoch 00084: val_loss did not improve
 - 1s - loss: 0.2607 - acc: 0.8844 - val_loss: 0.2826 - val_acc: 0.8598
Epoch 85/100
Epoch 00085: val_loss did not improve
 - 1s - loss: 0.2598 - acc: 0.8857 - val_loss: 0.2858 - val_acc: 0.8629
Epoch 86/100
Epoch 00086: val_loss improved from 0.27960 to 0.27906, saving model to best_m.h5
 - 1s - loss: 0.2447 - acc: 0.8914 - val_loss: 0.2791 - val_acc: 0.8660
Epoch 87/100
Epoch 00087: val_loss did not improve
 - 1s - loss: 0.2420 - acc: 0.8906 - val_loss: 0.2859 - val_acc: 0.8629
Epoch 88/100
Epoch 00088: val_loss did not improve
 - 1s - loss: 0.2526 - acc: 0.8880 - val_loss: 0.2828 - val_acc: 0.8660
Epoch 89/100
Epoch 00089: val_loss did not improve
 - 1s - loss: 0.2508 - acc: 0.8914 - val_loss: 0.2851 - val_acc: 0.8567
Epoch 90/100
Epoch 00090: val_loss did not improve
 - 1s - loss: 0.2561 - acc: 0.8883 - val_los

Epoch 44/100
Epoch 00044: val_loss did not improve
 - 1s - loss: 0.2590 - acc: 0.8859 - val_loss: 0.2641 - val_acc: 0.8692
Epoch 45/100
Epoch 00045: val_loss did not improve
 - 1s - loss: 0.2747 - acc: 0.8792 - val_loss: 0.2609 - val_acc: 0.8785
Epoch 46/100
Epoch 00046: val_loss improved from 0.25996 to 0.25649, saving model to best_m.h5
 - 1s - loss: 0.2714 - acc: 0.8828 - val_loss: 0.2565 - val_acc: 0.8754
Epoch 47/100
Epoch 00047: val_loss did not improve
 - 1s - loss: 0.2687 - acc: 0.8726 - val_loss: 0.2585 - val_acc: 0.8660
Epoch 48/100
Epoch 00048: val_loss did not improve
 - 1s - loss: 0.2903 - acc: 0.8625 - val_loss: 0.2663 - val_acc: 0.8629
Epoch 49/100
Epoch 00049: val_loss did not improve
 - 1s - loss: 0.2662 - acc: 0.8727 - val_loss: 0.2575 - val_acc: 0.8660
Epoch 50/100
Epoch 00050: val_loss did not improve
 - 1s - loss: 0.3122 - acc: 0.8544 - val_loss: 0.2578 - val_acc: 0.8723
Epoch 51/100
Epoch 00051: val_loss did not improve
 - 1s - loss: 0.2611 - acc: 0.8797 - val_los

Epoch 7/100
Epoch 00007: val_loss improved from 0.50288 to 0.42303, saving model to best_m.h5
 - 1s - loss: 0.4609 - acc: 0.7982 - val_loss: 0.4230 - val_acc: 0.8255
Epoch 8/100
Epoch 00008: val_loss did not improve
 - 1s - loss: 0.4567 - acc: 0.7992 - val_loss: 0.4742 - val_acc: 0.7632
Epoch 9/100
Epoch 00009: val_loss improved from 0.42303 to 0.40359, saving model to best_m.h5
 - 1s - loss: 0.4219 - acc: 0.8199 - val_loss: 0.4036 - val_acc: 0.8193
Epoch 10/100
Epoch 00010: val_loss did not improve
 - 1s - loss: 0.4427 - acc: 0.8094 - val_loss: 0.4250 - val_acc: 0.7944
Epoch 11/100
Epoch 00011: val_loss improved from 0.40359 to 0.38306, saving model to best_m.h5
 - 1s - loss: 0.4215 - acc: 0.7998 - val_loss: 0.3831 - val_acc: 0.8318
Epoch 12/100
Epoch 00012: val_loss improved from 0.38306 to 0.36309, saving model to best_m.h5
 - 1s - loss: 0.3695 - acc: 0.8383 - val_loss: 0.3631 - val_acc: 0.8474
Epoch 13/100
Epoch 00013: val_loss improved from 0.36309 to 0.32993, saving model to best

Epoch 69/100
Epoch 00069: val_loss did not improve
 - 1s - loss: 0.2583 - acc: 0.8828 - val_loss: 0.2688 - val_acc: 0.8972
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 1s - loss: 0.2653 - acc: 0.8859 - val_loss: 0.2732 - val_acc: 0.8879
Epoch 71/100
Epoch 00071: val_loss did not improve
 - 1s - loss: 0.2373 - acc: 0.8906 - val_loss: 0.2687 - val_acc: 0.8910
Epoch 72/100
Epoch 00072: val_loss did not improve
 - 1s - loss: 0.2594 - acc: 0.8841 - val_loss: 0.2633 - val_acc: 0.8972
Epoch 73/100
Epoch 00073: val_loss did not improve
 - 1s - loss: 0.2640 - acc: 0.8851 - val_loss: 0.2690 - val_acc: 0.8972
Epoch 74/100
Epoch 00074: val_loss did not improve
 - 1s - loss: 0.2516 - acc: 0.8812 - val_loss: 0.2650 - val_acc: 0.9034
Epoch 75/100
Epoch 00075: val_loss did not improve
 - 1s - loss: 0.2514 - acc: 0.8922 - val_loss: 0.2636 - val_acc: 0.8972
Epoch 76/100
Epoch 00076: val_loss did not improve
 - 1s - loss: 0.2420 - acc: 0.8953 - val_loss: 0.2661 - val_acc: 0.9003
Epoch 77/100
Epo

Epoch 32/100
Epoch 00032: val_loss did not improve
 - 1s - loss: 0.2588 - acc: 0.8750 - val_loss: 0.3439 - val_acc: 0.8500
Epoch 33/100
Epoch 00033: val_loss did not improve
 - 1s - loss: 0.2343 - acc: 0.8914 - val_loss: 0.3436 - val_acc: 0.8500
Epoch 34/100
Epoch 00034: val_loss did not improve
 - 1s - loss: 0.2509 - acc: 0.8844 - val_loss: 0.3586 - val_acc: 0.8438
Epoch 35/100
Epoch 00035: val_loss did not improve
 - 1s - loss: 0.2499 - acc: 0.8875 - val_loss: 0.3594 - val_acc: 0.8344
Epoch 36/100
Epoch 00036: val_loss did not improve
 - 1s - loss: 0.2640 - acc: 0.8680 - val_loss: 0.3513 - val_acc: 0.8594
Epoch 37/100
Epoch 00037: val_loss did not improve
 - 1s - loss: 0.2260 - acc: 0.9086 - val_loss: 0.3553 - val_acc: 0.8562
Epoch 38/100
Epoch 00038: val_loss did not improve
 - 1s - loss: 0.2275 - acc: 0.8953 - val_loss: 0.3578 - val_acc: 0.8406
Epoch 39/100
Epoch 00039: val_loss did not improve
 - 1s - loss: 0.2454 - acc: 0.8828 - val_loss: 0.3442 - val_acc: 0.8781
Epoch 40/100
Epo

Epoch 99/100
Epoch 00099: val_loss did not improve
 - 1s - loss: 0.2273 - acc: 0.9000 - val_loss: 0.3535 - val_acc: 0.8594
Epoch 100/100
Epoch 00100: val_loss did not improve
 - 1s - loss: 0.2119 - acc: 0.9023 - val_loss: 0.3600 - val_acc: 0.8469
0.272870503197


In [8]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))
    return model
print('model model')

model model


In [9]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=5)


with open('../features/cnn_3_aug_rescale_preprocess_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))

# this 2737

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.68779, saving model to best_m.h5
 - 3s - loss: 0.6926 - acc: 0.5507 - val_loss: 0.6878 - val_acc: 0.4891
Epoch 2/100
Epoch 00002: val_loss improved from 0.68779 to 0.62594, saving model to best_m.h5
 - 2s - loss: 0.6655 - acc: 0.5880 - val_loss: 0.6259 - val_acc: 0.6044
Epoch 3/100
Epoch 00003: val_loss improved from 0.62594 to 0.60938, saving model to best_m.h5
 - 2s - loss: 0.5904 - acc: 0.6708 - val_loss: 0.6094 - val_acc: 0.6916
Epoch 4/100
Epoch 00004: val_loss improved from 0.60938 to 0.54694, saving model to best_m.h5
 - 2s - loss: 0.5826 - acc: 0.6732 - val_loss: 0.5469 - val_acc: 0.7227
Epoch 5/100
Epoch 00005: val_loss did not improve
 - 2s - loss: 0.5340 - acc: 0.7297 - val_loss: 0.7280 - val_acc: 0.5607
Epoch 6/100
Epoch 00006: val_loss did not improve
 - 2s - loss: 0.5524 - acc: 0.7106 - val_loss: 0.5495 - val_acc: 0.7290
Epoch 7/100
Epoch 00007: val_loss improved from 0.54694 to 0.44305, saving model to best_m.h5
 -

Epoch 62/100
Epoch 00062: val_loss did not improve
 - 2s - loss: 0.3117 - acc: 0.8609 - val_loss: 0.2758 - val_acc: 0.8629
Epoch 63/100
Epoch 00063: val_loss improved from 0.27337 to 0.26688, saving model to best_m.h5
 - 2s - loss: 0.3111 - acc: 0.8609 - val_loss: 0.2669 - val_acc: 0.8847
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 2s - loss: 0.2904 - acc: 0.8664 - val_loss: 0.2867 - val_acc: 0.8505
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 2s - loss: 0.3023 - acc: 0.8578 - val_loss: 0.2740 - val_acc: 0.8754
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 2s - loss: 0.3254 - acc: 0.8515 - val_loss: 0.2882 - val_acc: 0.8505
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 2s - loss: 0.2990 - acc: 0.8615 - val_loss: 0.2786 - val_acc: 0.8692
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.3134 - acc: 0.8513 - val_loss: 0.2756 - val_acc: 0.8723
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 2s - loss: 0.2923 - acc: 0.8640 - val_los

Epoch 23/100
Epoch 00023: val_loss did not improve
 - 2s - loss: 0.3472 - acc: 0.8328 - val_loss: 0.3461 - val_acc: 0.8692
Epoch 24/100
Epoch 00024: val_loss did not improve
 - 2s - loss: 0.3457 - acc: 0.8443 - val_loss: 0.3715 - val_acc: 0.8255
Epoch 25/100
Epoch 00025: val_loss did not improve
 - 2s - loss: 0.3546 - acc: 0.8227 - val_loss: 0.3768 - val_acc: 0.8287
Epoch 26/100
Epoch 00026: val_loss did not improve
 - 2s - loss: 0.3692 - acc: 0.8187 - val_loss: 0.3803 - val_acc: 0.8318
Epoch 27/100
Epoch 00027: val_loss improved from 0.34410 to 0.34207, saving model to best_m.h5
 - 2s - loss: 0.3484 - acc: 0.8334 - val_loss: 0.3421 - val_acc: 0.8380
Epoch 28/100
Epoch 00028: val_loss did not improve
 - 2s - loss: 0.3336 - acc: 0.8388 - val_loss: 0.3940 - val_acc: 0.8287
Epoch 29/100
Epoch 00029: val_loss improved from 0.34207 to 0.33601, saving model to best_m.h5
 - 2s - loss: 0.3512 - acc: 0.8383 - val_loss: 0.3360 - val_acc: 0.8224
Epoch 30/100
Epoch 00030: val_loss did not improve


Epoch 85/100
Epoch 00085: val_loss improved from 0.29008 to 0.28896, saving model to best_m.h5
 - 2s - loss: 0.3095 - acc: 0.8492 - val_loss: 0.2890 - val_acc: 0.8598
Epoch 86/100
Epoch 00086: val_loss did not improve
 - 2s - loss: 0.2935 - acc: 0.8633 - val_loss: 0.2933 - val_acc: 0.8629
Epoch 87/100
Epoch 00087: val_loss did not improve
 - 2s - loss: 0.2696 - acc: 0.8820 - val_loss: 0.2970 - val_acc: 0.8598
Epoch 88/100
Epoch 00088: val_loss did not improve
 - 2s - loss: 0.3058 - acc: 0.8584 - val_loss: 0.3028 - val_acc: 0.8505
Epoch 89/100
Epoch 00089: val_loss did not improve
 - 2s - loss: 0.2892 - acc: 0.8664 - val_loss: 0.3048 - val_acc: 0.8536
Epoch 90/100
Epoch 00090: val_loss did not improve
 - 2s - loss: 0.2889 - acc: 0.8672 - val_loss: 0.2901 - val_acc: 0.8629
Epoch 91/100
Epoch 00091: val_loss did not improve
 - 2s - loss: 0.2998 - acc: 0.8680 - val_loss: 0.2966 - val_acc: 0.8598
Epoch 92/100
Epoch 00092: val_loss did not improve
 - 2s - loss: 0.3080 - acc: 0.8529 - val_los

Epoch 45/100
Epoch 00045: val_loss did not improve
 - 2s - loss: 0.3228 - acc: 0.8421 - val_loss: 0.2654 - val_acc: 0.8723
Epoch 46/100
Epoch 00046: val_loss improved from 0.26172 to 0.25939, saving model to best_m.h5
 - 2s - loss: 0.3193 - acc: 0.8672 - val_loss: 0.2594 - val_acc: 0.8847
Epoch 47/100
Epoch 00047: val_loss did not improve
 - 2s - loss: 0.3531 - acc: 0.8365 - val_loss: 0.2604 - val_acc: 0.8754
Epoch 48/100
Epoch 00048: val_loss did not improve
 - 2s - loss: 0.2991 - acc: 0.8656 - val_loss: 0.2633 - val_acc: 0.8816
Epoch 49/100
Epoch 00049: val_loss did not improve
 - 2s - loss: 0.3246 - acc: 0.8492 - val_loss: 0.2599 - val_acc: 0.8785
Epoch 50/100
Epoch 00050: val_loss improved from 0.25939 to 0.25838, saving model to best_m.h5
 - 2s - loss: 0.3268 - acc: 0.8511 - val_loss: 0.2584 - val_acc: 0.8847
Epoch 51/100
Epoch 00051: val_loss did not improve
 - 2s - loss: 0.3423 - acc: 0.8359 - val_loss: 0.2585 - val_acc: 0.8785
Epoch 52/100
Epoch 00052: val_loss did not improve


Epoch 6/100
Epoch 00006: val_loss did not improve
 - 2s - loss: 0.5896 - acc: 0.6701 - val_loss: 0.6062 - val_acc: 0.6885
Epoch 7/100
Epoch 00007: val_loss did not improve
 - 2s - loss: 0.5798 - acc: 0.6813 - val_loss: 0.6539 - val_acc: 0.5701
Epoch 8/100
Epoch 00008: val_loss improved from 0.59000 to 0.58272, saving model to best_m.h5
 - 2s - loss: 0.5830 - acc: 0.6641 - val_loss: 0.5827 - val_acc: 0.6604
Epoch 9/100
Epoch 00009: val_loss did not improve
 - 2s - loss: 0.5567 - acc: 0.6982 - val_loss: 0.5848 - val_acc: 0.6636
Epoch 10/100
Epoch 00010: val_loss improved from 0.58272 to 0.55055, saving model to best_m.h5
 - 2s - loss: 0.5847 - acc: 0.6781 - val_loss: 0.5506 - val_acc: 0.6636
Epoch 11/100
Epoch 00011: val_loss improved from 0.55055 to 0.54230, saving model to best_m.h5
 - 2s - loss: 0.5590 - acc: 0.6954 - val_loss: 0.5423 - val_acc: 0.6978
Epoch 12/100
Epoch 00012: val_loss improved from 0.54230 to 0.52198, saving model to best_m.h5
 - 2s - loss: 0.5253 - acc: 0.7265 - va

Epoch 67/100
Epoch 00067: val_loss did not improve
 - 2s - loss: 0.3043 - acc: 0.8664 - val_loss: 0.2713 - val_acc: 0.8785
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.3122 - acc: 0.8445 - val_loss: 0.2646 - val_acc: 0.8816
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 2s - loss: 0.3079 - acc: 0.8640 - val_loss: 0.2751 - val_acc: 0.8629
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 2s - loss: 0.3252 - acc: 0.8419 - val_loss: 0.2800 - val_acc: 0.8629
Epoch 71/100
Epoch 00071: val_loss did not improve
 - 2s - loss: 0.2954 - acc: 0.8513 - val_loss: 0.2688 - val_acc: 0.8723
Epoch 72/100
Epoch 00072: val_loss did not improve
 - 2s - loss: 0.3013 - acc: 0.8656 - val_loss: 0.2661 - val_acc: 0.8723
Epoch 73/100
Epoch 00073: val_loss improved from 0.26416 to 0.25776, saving model to best_m.h5
 - 2s - loss: 0.3221 - acc: 0.8547 - val_loss: 0.2578 - val_acc: 0.8785
Epoch 74/100
Epoch 00074: val_loss did not improve
 - 2s - loss: 0.2948 - acc: 0.8586 - val_los

Epoch 28/100
Epoch 00028: val_loss did not improve
 - 2s - loss: 0.3818 - acc: 0.8343 - val_loss: 0.3680 - val_acc: 0.8313
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 2s - loss: 0.3744 - acc: 0.8242 - val_loss: 0.3882 - val_acc: 0.8313
Epoch 30/100
Epoch 00030: val_loss improved from 0.34386 to 0.33539, saving model to best_m.h5
 - 2s - loss: 0.3828 - acc: 0.8274 - val_loss: 0.3354 - val_acc: 0.8688
Epoch 31/100
Epoch 00031: val_loss did not improve
 - 2s - loss: 0.3465 - acc: 0.8367 - val_loss: 0.3434 - val_acc: 0.8375
Epoch 32/100
Epoch 00032: val_loss did not improve
 - 2s - loss: 0.3450 - acc: 0.8297 - val_loss: 0.3457 - val_acc: 0.8562
Epoch 33/100
Epoch 00033: val_loss improved from 0.33539 to 0.33005, saving model to best_m.h5
 - 2s - loss: 0.3364 - acc: 0.8446 - val_loss: 0.3300 - val_acc: 0.8531
Epoch 34/100
Epoch 00034: val_loss did not improve
 - 2s - loss: 0.3278 - acc: 0.8547 - val_loss: 0.3360 - val_acc: 0.8531
Epoch 35/100
Epoch 00035: val_loss did not improve


Epoch 94/100
Epoch 00094: val_loss did not improve
 - 2s - loss: 0.2830 - acc: 0.8641 - val_loss: 0.3312 - val_acc: 0.8438
Epoch 95/100
Epoch 00095: val_loss did not improve
 - 2s - loss: 0.2845 - acc: 0.8750 - val_loss: 0.3245 - val_acc: 0.8469
Epoch 96/100
Epoch 00096: val_loss did not improve
 - 2s - loss: 0.3068 - acc: 0.8633 - val_loss: 0.3569 - val_acc: 0.8313
Epoch 97/100
Epoch 00097: val_loss did not improve
 - 2s - loss: 0.3202 - acc: 0.8437 - val_loss: 0.3606 - val_acc: 0.8281
Epoch 98/100
Epoch 00098: val_loss did not improve
 - 2s - loss: 0.2973 - acc: 0.8539 - val_loss: 0.3521 - val_acc: 0.8344
Epoch 99/100
Epoch 00099: val_loss did not improve
 - 2s - loss: 0.3084 - acc: 0.8633 - val_loss: 0.3921 - val_acc: 0.8125
Epoch 100/100
Epoch 00100: val_loss did not improve
 - 2s - loss: 0.3131 - acc: 0.8445 - val_loss: 0.3548 - val_acc: 0.8344
0.273781292036


In [10]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
def ConvBlock(model, layers, filters):
    '''Create [layers] layers consisting of zero padding, a convolution with [filters] 3x3 filters and batch normalization. Perform max pooling after the last layer.'''
    for i in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Conv2D(filters, (3, 3), activation='relu'))
        model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()

    # Input image: 75x75x3
    model.add(Lambda(lambda x: x, input_shape=(75, 75, 3)))
    ConvBlock(model, 1, 32)
    # 37x37x32
    ConvBlock(model, 1, 64)
    # 18x18x64
    ConvBlock(model, 1, 128)
    # 9x9x128
    ConvBlock(model, 1, 128)
    # 4x4x128
    model.add(Flatten())
    model.add(Dense(1,activation='sigmoid'))
    
    return model
print('model model')


model model


In [11]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=5)


with open('../features/cnn_4_aug_rescale_preprocess_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))

# this 2146

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.75859, saving model to best_m.h5
 - 4s - loss: 0.6641 - acc: 0.7015 - val_loss: 0.7586 - val_acc: 0.4984
Epoch 2/100
Epoch 00002: val_loss improved from 0.75859 to 0.64598, saving model to best_m.h5
 - 2s - loss: 0.4345 - acc: 0.7974 - val_loss: 0.6460 - val_acc: 0.6137
Epoch 3/100
Epoch 00003: val_loss did not improve
 - 2s - loss: 0.4589 - acc: 0.7902 - val_loss: 0.6541 - val_acc: 0.5265
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 2s - loss: 0.4229 - acc: 0.8058 - val_loss: 1.3581 - val_acc: 0.4984
Epoch 5/100
Epoch 00005: val_loss improved from 0.64598 to 0.45685, saving model to best_m.h5
 - 2s - loss: 0.3829 - acc: 0.8117 - val_loss: 0.4569 - val_acc: 0.8193
Epoch 6/100
Epoch 00006: val_loss improved from 0.45685 to 0.41373, saving model to best_m.h5
 - 2s - loss: 0.3701 - acc: 0.8326 - val_loss: 0.4137 - val_acc: 0.8100
Epoch 7/100
Epoch 00007: val_loss did not improve
 - 2s - loss: 0.4174 - acc: 0.8130 - val_loss:

Epoch 00063: val_loss did not improve
 - 2s - loss: 0.1597 - acc: 0.9359 - val_loss: 0.1980 - val_acc: 0.9159
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 2s - loss: 0.1756 - acc: 0.9232 - val_loss: 0.1892 - val_acc: 0.9128
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 2s - loss: 0.1725 - acc: 0.9312 - val_loss: 0.1876 - val_acc: 0.9252
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 2s - loss: 0.1726 - acc: 0.9258 - val_loss: 0.2008 - val_acc: 0.9065
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 2s - loss: 0.1765 - acc: 0.9344 - val_loss: 0.1853 - val_acc: 0.9252
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.1681 - acc: 0.9258 - val_loss: 0.1999 - val_acc: 0.9159
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 2s - loss: 0.1680 - acc: 0.9242 - val_loss: 0.2196 - val_acc: 0.8879
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 2s - loss: 0.1686 - acc: 0.9248 - val_loss: 0.2002 - val_acc: 0.9034
Epoch 71/100
Epoch 00071: val

Epoch 28/100
Epoch 00028: val_loss did not improve
 - 2s - loss: 0.2922 - acc: 0.8675 - val_loss: 0.4085 - val_acc: 0.8255
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 2s - loss: 0.2847 - acc: 0.8617 - val_loss: 0.3422 - val_acc: 0.8629
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 2s - loss: 0.2456 - acc: 0.8890 - val_loss: 0.4313 - val_acc: 0.7975
Epoch 31/100
Epoch 00031: val_loss did not improve
 - 2s - loss: 0.2213 - acc: 0.9102 - val_loss: 0.2646 - val_acc: 0.8785
Epoch 32/100
Epoch 00032: val_loss improved from 0.25879 to 0.23572, saving model to best_m.h5
 - 2s - loss: 0.2177 - acc: 0.9008 - val_loss: 0.2357 - val_acc: 0.8941
Epoch 33/100
Epoch 00033: val_loss did not improve
 - 2s - loss: 0.2141 - acc: 0.9177 - val_loss: 0.2440 - val_acc: 0.8785
Epoch 34/100
Epoch 00034: val_loss did not improve
 - 2s - loss: 0.2204 - acc: 0.9109 - val_loss: 0.2508 - val_acc: 0.8816
Epoch 35/100
Epoch 00035: val_loss did not improve
 - 2s - loss: 0.2006 - acc: 0.9242 - val_los

Epoch 94/100
Epoch 00094: val_loss did not improve
 - 2s - loss: 0.1449 - acc: 0.9391 - val_loss: 0.2572 - val_acc: 0.8816
Epoch 95/100
Epoch 00095: val_loss did not improve
 - 2s - loss: 0.1823 - acc: 0.9242 - val_loss: 0.2660 - val_acc: 0.8816
Epoch 96/100
Epoch 00096: val_loss did not improve
 - 2s - loss: 0.1483 - acc: 0.9414 - val_loss: 0.2385 - val_acc: 0.8847
Epoch 97/100
Epoch 00097: val_loss did not improve
 - 2s - loss: 0.1491 - acc: 0.9398 - val_loss: 0.2475 - val_acc: 0.8910
Epoch 98/100
Epoch 00098: val_loss did not improve
 - 2s - loss: 0.1456 - acc: 0.9351 - val_loss: 0.2647 - val_acc: 0.8879
Epoch 99/100
Epoch 00099: val_loss did not improve
 - 2s - loss: 0.1455 - acc: 0.9437 - val_loss: 0.2702 - val_acc: 0.8910
Epoch 100/100
Epoch 00100: val_loss did not improve
 - 2s - loss: 0.1729 - acc: 0.9240 - val_loss: 0.2711 - val_acc: 0.8847
Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.69894, saving model to best_m.h5
 - 4s - loss: 0.7583 - acc: 0.7037 - val_loss: 0

Epoch 55/100
Epoch 00055: val_loss did not improve
 - 2s - loss: 0.1551 - acc: 0.9414 - val_loss: 0.2032 - val_acc: 0.9065
Epoch 56/100
Epoch 00056: val_loss did not improve
 - 2s - loss: 0.1763 - acc: 0.9172 - val_loss: 0.2121 - val_acc: 0.9097
Epoch 57/100
Epoch 00057: val_loss did not improve
 - 2s - loss: 0.1527 - acc: 0.9359 - val_loss: 0.2035 - val_acc: 0.9097
Epoch 58/100
Epoch 00058: val_loss did not improve
 - 2s - loss: 0.1895 - acc: 0.9211 - val_loss: 0.2038 - val_acc: 0.9097
Epoch 59/100
Epoch 00059: val_loss did not improve
 - 2s - loss: 0.1628 - acc: 0.9398 - val_loss: 0.2122 - val_acc: 0.8972
Epoch 60/100
Epoch 00060: val_loss did not improve
 - 2s - loss: 0.1732 - acc: 0.9320 - val_loss: 0.2114 - val_acc: 0.9128
Epoch 61/100
Epoch 00061: val_loss did not improve
 - 2s - loss: 0.1549 - acc: 0.9336 - val_loss: 0.1987 - val_acc: 0.9190
Epoch 62/100
Epoch 00062: val_loss did not improve
 - 2s - loss: 0.1885 - acc: 0.9226 - val_loss: 0.2206 - val_acc: 0.9097
Epoch 63/100
Epo

Epoch 20/100
Epoch 00020: val_loss did not improve
 - 2s - loss: 0.2813 - acc: 0.8734 - val_loss: 0.3073 - val_acc: 0.8723
Epoch 21/100
Epoch 00021: val_loss did not improve
 - 2s - loss: 0.2777 - acc: 0.8734 - val_loss: 0.3259 - val_acc: 0.8380
Epoch 22/100
Epoch 00022: val_loss did not improve
 - 2s - loss: 0.2809 - acc: 0.8750 - val_loss: 0.3622 - val_acc: 0.8255
Epoch 23/100
Epoch 00023: val_loss did not improve
 - 2s - loss: 0.2418 - acc: 0.8984 - val_loss: 0.2760 - val_acc: 0.8910
Epoch 24/100
Epoch 00024: val_loss did not improve
 - 2s - loss: 0.2470 - acc: 0.8937 - val_loss: 0.3910 - val_acc: 0.8255
Epoch 25/100
Epoch 00025: val_loss did not improve
 - 2s - loss: 0.2556 - acc: 0.8912 - val_loss: 0.5192 - val_acc: 0.7726
Epoch 26/100
Epoch 00026: val_loss did not improve
 - 2s - loss: 0.2469 - acc: 0.8953 - val_loss: 0.3364 - val_acc: 0.8442
Epoch 27/100
Epoch 00027: val_loss did not improve
 - 2s - loss: 0.2599 - acc: 0.8859 - val_loss: 0.2714 - val_acc: 0.8692
Epoch 28/100
Epo

Epoch 86/100
Epoch 00086: val_loss did not improve
 - 2s - loss: 0.1656 - acc: 0.9365 - val_loss: 0.2053 - val_acc: 0.9283
Epoch 87/100
Epoch 00087: val_loss did not improve
 - 2s - loss: 0.1484 - acc: 0.9367 - val_loss: 0.2291 - val_acc: 0.9034
Epoch 88/100
Epoch 00088: val_loss did not improve
 - 2s - loss: 0.1829 - acc: 0.9230 - val_loss: 0.2280 - val_acc: 0.8910
Epoch 89/100
Epoch 00089: val_loss did not improve
 - 2s - loss: 0.1795 - acc: 0.9154 - val_loss: 0.1979 - val_acc: 0.9190
Epoch 90/100
Epoch 00090: val_loss did not improve
 - 2s - loss: 0.1584 - acc: 0.9414 - val_loss: 0.1980 - val_acc: 0.9283
Epoch 91/100
Epoch 00091: val_loss did not improve
 - 2s - loss: 0.1741 - acc: 0.9310 - val_loss: 0.2257 - val_acc: 0.9190
Epoch 92/100
Epoch 00092: val_loss did not improve
 - 2s - loss: 0.1643 - acc: 0.9289 - val_loss: 0.1985 - val_acc: 0.9159
Epoch 93/100
Epoch 00093: val_loss did not improve
 - 2s - loss: 0.1637 - acc: 0.9328 - val_loss: 0.1960 - val_acc: 0.9159
Epoch 94/100
Epo

Epoch 51/100
Epoch 00051: val_loss did not improve
 - 2s - loss: 0.1647 - acc: 0.9281 - val_loss: 0.3663 - val_acc: 0.8469
Epoch 52/100
Epoch 00052: val_loss did not improve
 - 2s - loss: 0.1658 - acc: 0.9273 - val_loss: 0.3547 - val_acc: 0.8750
Epoch 53/100
Epoch 00053: val_loss did not improve
 - 2s - loss: 0.1478 - acc: 0.9430 - val_loss: 0.3593 - val_acc: 0.8531
Epoch 54/100
Epoch 00054: val_loss did not improve
 - 2s - loss: 0.1397 - acc: 0.9391 - val_loss: 0.3455 - val_acc: 0.8562
Epoch 55/100
Epoch 00055: val_loss did not improve
 - 2s - loss: 0.1596 - acc: 0.9328 - val_loss: 0.3666 - val_acc: 0.8531
Epoch 56/100
Epoch 00056: val_loss did not improve
 - 2s - loss: 0.1518 - acc: 0.9320 - val_loss: 0.3458 - val_acc: 0.8719
Epoch 57/100
Epoch 00057: val_loss did not improve
 - 2s - loss: 0.1666 - acc: 0.9282 - val_loss: 0.3459 - val_acc: 0.8688
Epoch 58/100
Epoch 00058: val_loss did not improve
 - 2s - loss: 0.1562 - acc: 0.9391 - val_loss: 0.3494 - val_acc: 0.8625
Epoch 59/100
Epo

In [12]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3), input_shape=(75, 75, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.3))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.3))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))
    return model
print('model model')


model model


In [13]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=5)


with open('../features/cnn_5_aug_rescale_preprocess_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
# 2364

Epoch 1/100
Epoch 00001: val_loss improved from inf to 1.05748, saving model to best_m.h5
 - 7s - loss: 0.6200 - acc: 0.6789 - val_loss: 1.0575 - val_acc: 0.4984
Epoch 2/100
Epoch 00002: val_loss improved from 1.05748 to 0.91963, saving model to best_m.h5
 - 3s - loss: 0.5291 - acc: 0.7224 - val_loss: 0.9196 - val_acc: 0.4984
Epoch 3/100
Epoch 00003: val_loss did not improve
 - 3s - loss: 0.4624 - acc: 0.7679 - val_loss: 1.3064 - val_acc: 0.4984
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 3s - loss: 0.4428 - acc: 0.7763 - val_loss: 1.3072 - val_acc: 0.4984
Epoch 5/100
Epoch 00005: val_loss did not improve
 - 3s - loss: 0.4065 - acc: 0.7992 - val_loss: 0.9701 - val_acc: 0.4953
Epoch 6/100
Epoch 00006: val_loss improved from 0.91963 to 0.53863, saving model to best_m.h5
 - 3s - loss: 0.4487 - acc: 0.7717 - val_loss: 0.5386 - val_acc: 0.6916
Epoch 7/100
Epoch 00007: val_loss did not improve
 - 3s - loss: 0.3915 - acc: 0.8187 - val_loss: 0.6422 - val_acc: 0.6698
Epoch 8/100
Epoch 

Epoch 65/100
Epoch 00065: val_loss did not improve
 - 3s - loss: 0.2648 - acc: 0.8763 - val_loss: 0.2363 - val_acc: 0.8785
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 3s - loss: 0.2831 - acc: 0.8719 - val_loss: 0.2156 - val_acc: 0.9003
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 3s - loss: 0.2583 - acc: 0.8906 - val_loss: 0.2154 - val_acc: 0.9003
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 3s - loss: 0.2592 - acc: 0.8867 - val_loss: 0.2195 - val_acc: 0.8879
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 3s - loss: 0.2748 - acc: 0.8719 - val_loss: 0.2318 - val_acc: 0.8754
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 3s - loss: 0.2497 - acc: 0.8953 - val_loss: 0.2259 - val_acc: 0.8847
Epoch 71/100
Epoch 00071: val_loss improved from 0.20535 to 0.20423, saving model to best_m.h5
 - 3s - loss: 0.2605 - acc: 0.8802 - val_loss: 0.2042 - val_acc: 0.9065
Epoch 72/100
Epoch 00072: val_loss did not improve
 - 3s - loss: 0.2761 - acc: 0.8810 - val_los

Epoch 29/100
Epoch 00029: val_loss did not improve
 - 3s - loss: 0.3214 - acc: 0.8555 - val_loss: 0.2950 - val_acc: 0.8536
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 3s - loss: 0.3253 - acc: 0.8466 - val_loss: 0.3196 - val_acc: 0.8318
Epoch 31/100
Epoch 00031: val_loss improved from 0.27840 to 0.26891, saving model to best_m.h5
 - 3s - loss: 0.3053 - acc: 0.8625 - val_loss: 0.2689 - val_acc: 0.8754
Epoch 32/100
Epoch 00032: val_loss did not improve
 - 3s - loss: 0.2839 - acc: 0.8594 - val_loss: 0.2912 - val_acc: 0.8536
Epoch 33/100
Epoch 00033: val_loss did not improve
 - 3s - loss: 0.2920 - acc: 0.8531 - val_loss: 0.2903 - val_acc: 0.8474
Epoch 34/100
Epoch 00034: val_loss did not improve
 - 3s - loss: 0.3104 - acc: 0.8568 - val_loss: 0.2849 - val_acc: 0.8660
Epoch 35/100
Epoch 00035: val_loss did not improve
 - 3s - loss: 0.3074 - acc: 0.8566 - val_loss: 0.2812 - val_acc: 0.8723
Epoch 36/100
Epoch 00036: val_loss improved from 0.26891 to 0.26781, saving model to best_m.h5


Epoch 93/100
Epoch 00093: val_loss did not improve
 - 3s - loss: 0.2645 - acc: 0.8818 - val_loss: 0.2689 - val_acc: 0.8754
Epoch 94/100
Epoch 00094: val_loss did not improve
 - 3s - loss: 0.2753 - acc: 0.8763 - val_loss: 0.2621 - val_acc: 0.8692
Epoch 95/100
Epoch 00095: val_loss did not improve
 - 3s - loss: 0.2723 - acc: 0.8758 - val_loss: 0.2420 - val_acc: 0.8879
Epoch 96/100
Epoch 00096: val_loss improved from 0.23996 to 0.23866, saving model to best_m.h5
 - 3s - loss: 0.2614 - acc: 0.8812 - val_loss: 0.2387 - val_acc: 0.8941
Epoch 97/100
Epoch 00097: val_loss improved from 0.23866 to 0.23752, saving model to best_m.h5
 - 4s - loss: 0.2689 - acc: 0.8765 - val_loss: 0.2375 - val_acc: 0.8972
Epoch 98/100
Epoch 00098: val_loss did not improve
 - 4s - loss: 0.2758 - acc: 0.8677 - val_loss: 0.2387 - val_acc: 0.8910
Epoch 99/100
Epoch 00099: val_loss did not improve
 - 4s - loss: 0.2750 - acc: 0.8719 - val_loss: 0.2399 - val_acc: 0.9065
Epoch 100/100
Epoch 00100: val_loss improved from 0

Epoch 54/100
Epoch 00054: val_loss did not improve
 - 3s - loss: 0.2683 - acc: 0.8734 - val_loss: 0.2340 - val_acc: 0.8754
Epoch 55/100
Epoch 00055: val_loss did not improve
 - 3s - loss: 0.2868 - acc: 0.8677 - val_loss: 0.2355 - val_acc: 0.9065
Epoch 56/100
Epoch 00056: val_loss did not improve
 - 4s - loss: 0.2734 - acc: 0.8680 - val_loss: 0.2340 - val_acc: 0.8941
Epoch 57/100
Epoch 00057: val_loss did not improve
 - 4s - loss: 0.2886 - acc: 0.8730 - val_loss: 0.2323 - val_acc: 0.8754
Epoch 58/100
Epoch 00058: val_loss did not improve
 - 3s - loss: 0.2861 - acc: 0.8826 - val_loss: 0.2409 - val_acc: 0.8816
Epoch 59/100
Epoch 00059: val_loss improved from 0.23140 to 0.22903, saving model to best_m.h5
 - 4s - loss: 0.2830 - acc: 0.8719 - val_loss: 0.2290 - val_acc: 0.9003
Epoch 60/100
Epoch 00060: val_loss improved from 0.22903 to 0.22422, saving model to best_m.h5
 - 4s - loss: 0.2670 - acc: 0.8758 - val_loss: 0.2242 - val_acc: 0.8879
Epoch 61/100
Epoch 00061: val_loss did not improve


Epoch 18/100
Epoch 00018: val_loss did not improve
 - 3s - loss: 0.3501 - acc: 0.8359 - val_loss: 0.6502 - val_acc: 0.6947
Epoch 19/100
Epoch 00019: val_loss did not improve
 - 3s - loss: 0.3434 - acc: 0.8429 - val_loss: 0.3676 - val_acc: 0.8224
Epoch 20/100
Epoch 00020: val_loss improved from 0.32899 to 0.29447, saving model to best_m.h5
 - 4s - loss: 0.3172 - acc: 0.8539 - val_loss: 0.2945 - val_acc: 0.8474
Epoch 21/100
Epoch 00021: val_loss improved from 0.29447 to 0.28645, saving model to best_m.h5
 - 3s - loss: 0.3958 - acc: 0.8260 - val_loss: 0.2865 - val_acc: 0.8910
Epoch 22/100
Epoch 00022: val_loss did not improve
 - 3s - loss: 0.3361 - acc: 0.8558 - val_loss: 0.3050 - val_acc: 0.8442
Epoch 23/100
Epoch 00023: val_loss did not improve
 - 3s - loss: 0.3654 - acc: 0.8365 - val_loss: 0.3318 - val_acc: 0.8349
Epoch 24/100
Epoch 00024: val_loss did not improve
 - 3s - loss: 0.3229 - acc: 0.8500 - val_loss: 0.5994 - val_acc: 0.7072
Epoch 25/100
Epoch 00025: val_loss did not improve


Epoch 83/100
Epoch 00083: val_loss did not improve
 - 3s - loss: 0.2838 - acc: 0.8738 - val_loss: 0.2339 - val_acc: 0.8941
Epoch 84/100
Epoch 00084: val_loss did not improve
 - 3s - loss: 0.2773 - acc: 0.8883 - val_loss: 0.2268 - val_acc: 0.9065
Epoch 85/100
Epoch 00085: val_loss did not improve
 - 3s - loss: 0.2703 - acc: 0.8875 - val_loss: 0.2235 - val_acc: 0.9065
Epoch 86/100
Epoch 00086: val_loss did not improve
 - 3s - loss: 0.2776 - acc: 0.8888 - val_loss: 0.2282 - val_acc: 0.9159
Epoch 87/100
Epoch 00087: val_loss did not improve
 - 3s - loss: 0.2679 - acc: 0.8742 - val_loss: 0.2217 - val_acc: 0.9065
Epoch 88/100
Epoch 00088: val_loss did not improve
 - 3s - loss: 0.2794 - acc: 0.8761 - val_loss: 0.2325 - val_acc: 0.9097
Epoch 89/100
Epoch 00089: val_loss did not improve
 - 3s - loss: 0.2654 - acc: 0.8732 - val_loss: 0.2267 - val_acc: 0.9034
Epoch 90/100
Epoch 00090: val_loss did not improve
 - 3s - loss: 0.2717 - acc: 0.8824 - val_loss: 0.2257 - val_acc: 0.9034
Epoch 91/100
Epo

Epoch 46/100
Epoch 00046: val_loss did not improve
 - 3s - loss: 0.2638 - acc: 0.8805 - val_loss: 0.3317 - val_acc: 0.8750
Epoch 47/100
Epoch 00047: val_loss did not improve
 - 3s - loss: 0.2671 - acc: 0.8930 - val_loss: 0.3586 - val_acc: 0.8500
Epoch 48/100
Epoch 00048: val_loss did not improve
 - 3s - loss: 0.2795 - acc: 0.8695 - val_loss: 0.3350 - val_acc: 0.8688
Epoch 49/100
Epoch 00049: val_loss did not improve
 - 3s - loss: 0.2676 - acc: 0.8758 - val_loss: 0.3420 - val_acc: 0.8656
Epoch 50/100
Epoch 00050: val_loss did not improve
 - 3s - loss: 0.2543 - acc: 0.8883 - val_loss: 0.3162 - val_acc: 0.8719
Epoch 51/100
Epoch 00051: val_loss did not improve
 - 3s - loss: 0.2974 - acc: 0.8648 - val_loss: 0.3325 - val_acc: 0.8625
Epoch 52/100
Epoch 00052: val_loss did not improve
 - 3s - loss: 0.2632 - acc: 0.8664 - val_loss: 0.3369 - val_acc: 0.8719
Epoch 53/100
Epoch 00053: val_loss did not improve
 - 3s - loss: 0.2584 - acc: 0.8813 - val_loss: 0.3303 - val_acc: 0.8719
Epoch 54/100
Epo